In [1]:
import tensorflow as tf
import numpy as np
from itertools import groupby

sentences = [
    ['dave', 'smith', 'went', 'to', 'the', 'church', 'and', 'talked', 'mary', 'into', 'doing', 'it'],
    ['john', 'is', 'a', 'magnificent', 'scotsman', 'that', 'drinks', 'whisky', 'by', 'the', 'lot'],
    ['evan', 'connor', 'won', 'the', 'tennis', 'match', 'against', 'erick', 'smith']
]
actual_labels = [
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 1, 1]
]

vocab = list(set(t for s in sentences for t in s))
vocab_size = len(vocab)
embeddings_size = 15

words = tf.placeholder(tf.string, shape=(None,), name='words')
label_ids = tf.placeholder(tf.int32, shape=(None,), name='label_ids')
labels = tf.one_hot(label_ids, 2)

mapping_vocab = tf.constant(vocab)
table = tf.contrib.lookup.index_table_from_tensor(mapping=mapping_vocab)
word_ids = table.lookup(words)

with tf.variable_scope('x', reuse=tf.AUTO_REUSE):
    v = tf.get_variable('embeddings', [vocab_size, embeddings_size], tf.float32)
    embeddings = tf.nn.embedding_lookup(v, word_ids)

    output = tf.layers.dense(embeddings, 10)
    logits = tf.layers.dense(output, 2)
    pred_ids = tf.argmax(logits, axis=-1)
    
    probs = tf.nn.softmax(logits)
    a_tilde = probs[:,1]    
    a_tilde = tf.reduce_sum(tf.boolean_mask(a_tilde, label_ids))    
    n_pos = tf.reduce_sum(tf.cast(label_ids, dtype=tf.float32))
    m_pos_tilde = tf.reduce_sum(probs[:,1]) 
    
    alpha = 0.5
    loss = -tf.divide(a_tilde, (alpha * n_pos + (1 - alpha) * m_pos_tilde))
    train_step = tf.train.AdamOptimizer().minimize(loss)

    correct = tf.equal(tf.to_int32(pred_ids), label_ids)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits))
    # train_step = tf.train.AdamOptimizer().minimize(loss)
   
with tf.Session() as sess:
    tf.tables_initializer().run()
    tf.initializers.global_variables().run()
    
    for _ in range(1000):
        a, f = [], []
        for i, s in enumerate(sentences):
            f1, acc, preds, _ = sess.run([loss, accuracy, pred_ids, train_step], feed_dict={
                words: [t for t in s],
                label_ids: actual_labels[i]
            })
            a.append(acc)
            f.append(f1)

        a = sum(a) / 3 
        f = sum(f) / 3
        print(a, f)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.361952866117 -0.324544370174
0.368686874708 -0.329447244604
0.368686874708 -0.334059273203
0.433501680692 -0.338632658124
0.461279471715 -0.343191762765
0.461279471715 -0.347751339277
0.498316516479 -0.352326869965
0.498316516479 -0.356933822234
0.498316516479 -0.361586709817
0.526094287634 -0.366298576196
0.526094287634 -0.371082037687
0.55639731884 -0.375949387749
0.55639731884 -0.380912944674
0.55639731884 -0.385985160867
0.55639731884 -0.391178602974
0.55639731884 -0.396506225069
0.55639731884 -0.401981299122
0.55639731884 -0.407617767652
0.55639731884 -0.413430338105
0.55639731884 -0.419434751074
0.586700350046 -0.425648123026
0.623737384876 -0.432088961204
0.623737384876 -0.43877766033
0.623737384876 -0.445736944675
0.749158243338 -0.452991505464
0.779461284479 -0.460568855206
0.809764305751 -0.46849907438
0.809764305751 -0.476814786593
0.809764305751 -0.485551158587
0.867845137914 -0.494745264451
0.867845137914 -0.504435797532
0.895622909069 -0.514661659797
0.932659943899 -0.5

1.0 -0.999707142512
1.0 -0.999708712101
1.0 -0.999710261822
1.0 -0.999711771806
1.0 -0.999713261922
1.0 -0.999714791775
1.0 -0.999716281891
1.0 -0.999717732271
1.0 -0.999719242255
1.0 -0.999720672766
1.0 -0.999722162882
1.0 -0.99972363313
1.0 -0.999725023905
1.0 -0.99972641468
1.0 -0.999727825324
1.0 -0.999729255835
1.0 -0.999730606874
1.0 -0.999732017517
1.0 -0.999733368556
1.0 -0.999734719594
1.0 -0.999736110369
1.0 -0.999737421672
1.0 -0.999738713106
1.0 -0.999740064144
1.0 -0.999741355578
1.0 -0.999742647012
1.0 -0.999743978182
1.0 -0.999745289485
1.0 -0.999746521314
1.0 -0.99974779288
1.0 -0.999749024709
1.0 -0.999750276407
1.0 -0.999751488368
1.0 -0.999752740065
1.0 -0.999753991763
1.0 -0.999755163987
1.0 -0.999756336212
1.0 -0.999757568041
1.0 -0.999758780003
1.0 -0.999759932359
1.0 -0.999761084716
1.0 -0.999762276808
1.0 -0.999763449033
1.0 -0.999764561653
1.0 -0.999765733878
1.0 -0.999766767025
1.0 -0.999767998854
1.0 -0.999769111474
1.0 -0.999770144622
1.0 -0.999771296978
1.0

1.0 -0.999944746494
1.0 -0.99994490544
1.0 -0.999945044518
1.0 -0.999945263068
1.0 -0.999945382277
1.0 -0.999945521355
1.0 -0.999945700169
1.0 -0.999945839246
1.0 -0.999945998192
1.0 -0.999946137269
1.0 -0.999946316083
1.0 -0.999946455161
1.0 -0.99994657437
1.0 -0.99994679292
1.0 -0.999946892262
1.0 -0.999947071075
1.0 -0.999947309494
1.0 -0.99994734923
1.0 -0.999947528044
1.0 -0.999947647254
1.0 -0.999947845936
1.0 -0.999947965145
1.0 -0.999948084354
1.0 -0.9999482433
1.0 -0.999948402246
1.0 -0.999948501587
1.0 -0.999948680401
1.0 -0.999948879083
1.0 -0.999948978424
1.0 -0.999949097633
1.0 -0.999949256579
1.0 -0.999949415525
1.0 -0.999949554602
1.0 -0.999949653943
1.0 -0.999949773153
1.0 -0.999949932098
1.0 -0.999950110912
1.0 -0.999950230122
1.0 -0.999950389067
1.0 -0.999950488408
1.0 -0.999950627486
1.0 -0.999950786432
1.0 -0.999950905641
1.0 -0.99995102485
1.0 -0.999951203664
1.0 -0.999951322873
1.0 -0.999951422215
1.0 -0.999951601028
1.0 -0.999951720238
1.0 -0.999951859315
1.0 -0.